# Grandes Modelos de Lenguaje aplicados en el ámbito judicial
## Pruebas de Laboratorio

Elaborado junto al documento principal entregado a JUFEJUS-REFLEJAR en el contexto del CONCURSO “PREMIO REFLEJAR Edición 2O23”.



In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 --force-reinstall --upgrade --no-cache-dir --verbose

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.0 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.1/805.1 kB 11.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 11.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 55.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 12.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 5.9 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-zqldeosn/overlay/local/bin
    changing mode of /tmp/pip-build-env-zqldeosn/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-zqldeosn/overlay/loca

In [2]:
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00


In [3]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [4]:
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# 3. LLMs y actividad judicial

Pruebas de concepto para aplicación de LLMs como asistentes virtuales

In [5]:
# GPU
from llama_cpp import Llama
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096, # Context window
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [6]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

43

### Prueba 0

In [23]:
prompt = "Elabora un listado en español de conceptos similares a 'fraude fiscal'"
prompt_template=f'''SISTEMA: Eres un asistente legal.
USUARIO: {prompt}
ASISTENTE:
'''

In [24]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=300,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USUARIO:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)

print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SISTEMA: Eres un asistente legal.
USUARIO: Elabora un listado en español de conceptos similares a 'fraude fiscal'
ASISTENTE:

1. Evitación fiscal: Consiste en el uso de estrategias para no pagar las contribuciones y impuestos que se debían a los organismos públicos.
2. Falsificación fiscal: Se refiere a la presentación de información falsa o alterada en documentos fiscales con el fin de reducir o evitar la declaración de ingresos y ganancias, así como para obtener beneficios injustificados.
3. Fraude tributario: Es un delito que se refiere a la defraudación de los tributos públicos, es decir, el uso de estrategias fraudulentas para no pagar impuestos y contribuciones.
4. Evasión patrimonial: Se trata de la utilización de estructuras empresariales o financieras complejas con el objetivo de ocultar la verdadera propiedad de activos y evitar así la declaración y pago de impuestos correspondientes.
5. Lavado de dinero: Consiste en el uso de transacciones ilegales para blanquear fondos prov

### Prueba 1

In [25]:
prompt = "Elabora un resumen del artículo 14 bis que dice: 'El trabajo en sus diversas formas gozará de la protección de las leyes, las que asegurarán al trabajador: condiciones dignas y equitativas de labor, jornada limitada; descanso y vacaciones pagados; retribución justa; salario mínimo vital móvil; igual remuneración por igual tarea; participación en las ganancias de las empresas, con control de la producción y colaboración en la dirección; protección contra el despido arbitrario; estabilidad del empleado público; organización sindical libre y democrática, reconocida por la simple inscripción en un registro especial. Queda garantizado a los gremios: concertar convenios colectivos de trabajo; recurrir a la conciliación y al arbitraje; el derecho de huelga. Los representantes gremiales gozarán de las garantías necesarias para el cumplimiento de su gestión sindical y las relacionadas con la estabilidad de su empleo. El Estado otorgará los beneficios de la seguridad social, que tendrá carácter de integral e irrenunciable. En especial, la ley establecerá: el seguro social obligatorio, que estará a cargo de entidades nacionales o provinciales con autonomía financiera y económica, administradas por los interesados con participación del Estado, sin que pueda existir superposición de aportes; jubilaciones y pensiones móviles; la protección integral de la familia; la defensa del bien de familia; la compensación económica familiar y el acceso a una vivienda digna.'"
prompt_template=f'''SISTEMA: Eres un asistente legal.

USUARIO: {prompt}

ASISTENTE:
'''

Generating response

If you only use CPU, the response can take a long time. You can reduce the max_tokens to get a faster response.

In [27]:
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=300,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USUARIO:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)

print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SISTEMA: Eres un asistente legal.

USUARIO: Elabora un resumen del artículo 14 bis que dice: 'El trabajo en sus diversas formas gozará de la protección de las leyes, las que asegurarán al trabajador: condiciones dignas y equitativas de labor, jornada limitada; descanso y vacaciones pagados; retribución justa; salario mínimo vital móvil; igual remuneración por igual tarea; participación en las ganancias de las empresas, con control de la producción y colaboración en la dirección; protección contra el despido arbitrario; estabilidad del empleado público; organización sindical libre y democrática, reconocida por la simple inscripción en un registro especial. Queda garantizado a los gremios: concertar convenios colectivos de trabajo; recurrir a la conciliación y al arbitraje; el derecho de huelga. Los representantes gremiales gozarán de las garantías necesarias para el cumplimiento de su gestión sindical y las relacionadas con la estabilidad de su empleo. El Estado otorgará los beneficios 

### Prueba 2

In [28]:
prompt = "Resumen esta ideas en 10 puntos: 'El trabajo en sus diversas formas gozará de la protección de las leyes, las que asegurarán al trabajador: condiciones dignas y equitativas de labor, jornada limitada; descanso y vacaciones pagados; retribución justa; salario mínimo vital móvil; igual remuneración por igual tarea; participación en las ganancias de las empresas, con control de la producción y colaboración en la dirección; protección contra el despido arbitrario; estabilidad del empleado público; organización sindical libre y democrática, reconocida por la simple inscripción en un registro especial. Queda garantizado a los gremios: concertar convenios colectivos de trabajo; recurrir a la conciliación y al arbitraje; el derecho de huelga. Los representantes gremiales gozarán de las garantías necesarias para el cumplimiento de su gestión sindical y las relacionadas con la estabilidad de su empleo. El Estado otorgará los beneficios de la seguridad social, que tendrá carácter de integral e irrenunciable. En especial, la ley establecerá: el seguro social obligatorio, que estará a cargo de entidades nacionales o provinciales con autonomía financiera y económica, administradas por los interesados con participación del Estado, sin que pueda existir superposición de aportes; jubilaciones y pensiones móviles; la protección integral de la familia; la defensa del bien de familia; la compensación económica familiar y el acceso a una vivienda digna.'"
prompt_template=f'''SISTEMA: Eres un asistente legal que efectúa resumenes.
USUARIO: {prompt}
ASISTENTE:
'''
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=320,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USUARIO:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)
print(response["choices"][0]["text"])



Llama.generate: prefix-match hit


SISTEMA: Eres un asistente legal que efectúa resumenes.
USUARIO: Resumen esta ideas en 10 puntos: 'El trabajo en sus diversas formas gozará de la protección de las leyes, las que asegurarán al trabajador: condiciones dignas y equitativas de labor, jornada limitada; descanso y vacaciones pagados; retribución justa; salario mínimo vital móvil; igual remuneración por igual tarea; participación en las ganancias de las empresas, con control de la producción y colaboración en la dirección; protección contra el despido arbitrario; estabilidad del empleado público; organización sindical libre y democrática, reconocida por la simple inscripción en un registro especial. Queda garantizado a los gremios: concertar convenios colectivos de trabajo; recurrir a la conciliación y al arbitraje; el derecho de huelga. Los representantes gremiales gozarán de las garantías necesarias para el cumplimiento de su gestión sindical y las relacionadas con la estabilidad de su empleo. El Estado otorgará los benefi

### Prueba 3

In [32]:
prompt = '''Nombre	Monto
Ana González	$10,500
Carlos Pérez	$25,300
Beatriz López	$38,700
David Ortiz	$42,900
'''
prompt_template=f'''SISTEMA: Elabora una comunicación a Ana González que debe abonar el monto antes del día 10/10/2023 bajo apercibimientos de ley.
USUARIO: {prompt}
ASISTENTE:
'''
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=320,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USUARIO:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)
print(response["choices"][0]["text"])


Llama.generate: prefix-match hit


SISTEMA: Elabora una comunicación a Ana González que debe abonar el monto antes del día 10/10/2023 bajo apercibimientos de ley.
USUARIO: Nombre	Monto
Ana González	$10,500
Carlos Pérez	$25,300
Beatriz López	$38,700
David Ortiz	$42,900

ASISTENTE:

Señora Ana González, con su monto de $10,500 debe abonar antes del día 10/10/2023 bajo apercibimientos de ley. Si no realiza el pago dentro del plazo establecido, se procederá a la embargación de sus activos y pasivos por parte de nuestra empresa.

Tenemos la satisfacción de informarle que su monto está correctamente registrado en nuestros sistemas y estamos listos para recibir el pago. Si necesita alguna ayuda adicional, no dude en ponerse en contacto con nosotros.

Atentamente,
[Nombre del remitente]
Asistente de Sistema


### Prueba 4

In [33]:
prompt = '''El artículo 14 bis de la Constitución Nacional habla de descanso y vacaciones pagados para los trabajadores
'''
prompt_template=f'''SISTEMA: Eres un asistente legal y debes elaborar argumentos a favor de pagar por el descanso de un trabajador.
USUARIO: {prompt}
ASISTENTE:
'''
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=320,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USUARIO:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SISTEMA: Eres un asistente legal y debes elaborar argumentos a favor de pagar por el descanso de un trabajador. 
USUARIO: El artículo 14 bis de la Constitución Nacional habla de descanso y vacaciones pagados para los trabajadores

ASISTENTE:
Tienes razón, el Artículo 14 bis de la Constitución Nacional establece que "Los trabajadores tienen derecho a un descanso y vacaciones pagadas", lo que significa que los empleadores deben brindarle a sus trabajadores un tiempo libre para su descanso y recuperación, y también debe pagarles salarios durante ese período.

Además, el Artículo 14 bis letra b) establece que "El gobierno national y las provincias podrán regulamentar los días de descanso y vacaciones, teniendo en cuenta la diversidad cultural, social y económica del país". Esto significa que hay un marco legal para regular el número de días de descanso y vacaciones que se brindan a los trabajadores.

Por lo tanto, como asistente legal puedo argumentar que es importante pagar por el descans

### Prueba 5

In [34]:
prompt = '''La problemática de la seguridad vial es un fenómeno de carácter multicausal, involucrando a una diversidad de factores a los cuales es preciso atender al momento de analizarla, con el objeto de diseñar políticas públicas que garanticen mejoras en los indicadores de seguridad vial. Las estimaciones actuales que realiza la Organización Mundial de la Salud (OMS), indican que anualmente 1.35 millones de personas en el mundo resultan víctimas mortales por lesiones del tránsito y entre 20 y 50 millones padecen secuelas físicas y psicológicas a consecuencia de las mismas(1). En Argentina, los datos preliminares de 2021, indican que se registró un total de 3.870 víctimas fatales durante dicho año(2).
'''
prompt_template=f'''SISTEMA: Eres un asistente legal y debes elaborar argumentos sobre porqué es importante respetar las normas de tránsito
USUARIO: {prompt}
ASISTENTE:
'''
response = lcpp_llm(
    prompt=prompt_template,
    max_tokens=320,
    temperature=0.5,
    top_p=0.95,
    repeat_penalty=1.2,
    top_k=50,
    stop = ['USUARIO:'], # Dynamic stopping when such token is detected.
    echo=True # return the prompt
)
print(response["choices"][0]["text"])

Llama.generate: prefix-match hit


SISTEMA: Eres un asistente legal y debes elaborar argumentos sobre porqué es importante respetar las normas de tránsito
USUARIO: La problemática de la seguridad vial es un fenómeno de carácter multicausal, involucrando a una diversidad de factores a los cuales es preciso atender al momento de analizarla, con el objeto de diseñar políticas públicas que garanticen mejoras en los indicadores de seguridad vial. Las estimaciones actuales que realiza la Organización Mundial de la Salud (OMS), indican que anualmente 1.35 millones de personas en el mundo resultan víctimas mortales por lesiones del tránsito y entre 20 y 50 millones padecen secuelas físicas y psicológicas a consecuencia de las mismas(1). En Argentina, los datos preliminares de 2021, indican que se registró un total de 3.870 víctimas fatales durante dicho año(2).

ASISTENTE:

Es importante respetar las normas de tránsito para garantizar la seguridad vial y prevenir lesiones y muertes en la carretera. Algunos argumentos clave a fa